In [1]:
#-----------------------------------------------------------Importing libraries----------------------------------------------------------------

import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer 
from sklearn.preprocessing import LabelEncoder 

#-----------------------------------------Preprocessing of raw training data-------------------------------------------------------------------

# Inputting csv file and replace missing data with blank space
df= pd.read_csv('trainData.csv')
df=df.replace('unknown', '')

# Separating features and class labels from training data
X_train=df.iloc[:,:-1].values
y_train=df.iloc[:,19].values

# Label Encoding of all the Categorical features in training data 

label_X = LabelEncoder()

for i in range (1,10):
  X_train[:,i] = label_X.fit_transform(X_train[:,i])
X_train[:,13] = label_X.fit_transform(X_train[:,13])


# Repalcing NaN values, if required,in features of training data, with the mean of column
imputer = Imputer(missing_values=0, strategy= 'mean',axis=0)
X_train[:,1:7]=imputer.fit_transform(X_train[:,1:7]) 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#-----------------------------------------Preprocessing of raw Test data---------------------------------------------------------------------

# Inputting csv file and replace missing data
df2 = pd.read_csv('testData.csv')
df2=df2.replace('unknown', '')

# Separating features and class labels from test data
X_test=df2.iloc[:,:-1].values
y_test=df2.iloc[:,19].values

# Label Encoding of all the non-numeric features in test data 

label_X1 = LabelEncoder()

for i in range (1,10):
  X_test[:,i] = label_X1.fit_transform(X_test[:,i])
X_test[:,13] = label_X1.fit_transform(X_test[:,13])

# Repalcing NaN values, in features of test data, with the mean of column
imputer = Imputer(missing_values=0, strategy= 'mean',axis=0)
X_test[:,1:7]=imputer.fit_transform(X_test[:,1:7]) 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
#---------------------------------XGBoost Classifier implemenation--------------------------------------------------------------------------
#importing libraries 
from xgboost import XGBClassifier
import xgboost as xgb

#Storing train and test data into the Dmatrix of XGboost 
dtrain=xgb.DMatrix(X_train, label=y_train)
dtest=xgb.DMatrix(X_test)

# Calculation of ratio of class with label 0 and class with label 1
train_labels= dtrain.get_label()
ratio= float(np.sum(train_labels==0))/np.sum(train_labels==1)

#Setting Parameters of XGBClassifer after cross validation 
classifier= XGBClassifier(
 learning_rate =0.0005,
 n_estimators=4500,
 max_depth=20,
 min_child_weight=20,
 max_delta_step=7,
 gamma=0.4,
 subsample=0.8,
 colsample_bytree=0.75,
 reg_alpha=0.01,
 reg_lambda=2,
 objective= 'binary:logistic',
 nthread=4,
 base_score=0.5,
 scale_pos_weight=ratio,
 seed=5000)

# Fitting the train data to XGBoost classifier 
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_test, y_pred)        
print('Confusion Matrix: ')               
print (cm)

#AUC calculation
from sklearn.metrics import roc_auc_score
AUC=roc_auc_score(y_test, y_pred)
print('Area under the ROC: ')
print (AUC)

Confusion Matrix: 
[[5551  815]
 [ 301  544]]
Area under the ROC: 
0.7578815527013888
